In [ ]:
# only run on google runtime
!pip install tensorflow-text
!pip install tf-models-official
!pip install tensorflow-addons
!pip install scikit-learn
!pip install transformers
!pip install scikit-learn

In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text
from official.nlp import optimization
import tensorflow_addons as tfa
import transformers
import sklearn as sk
from sklearn.model_selection import train_test_split

In [4]:
# only run on google runtime
# update paths to semeval task data on machine
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
input_directory = '/content/drive/MyDrive/2023-2024 School Year/Fall Semester/Natural Language Processing/Project/Data'
raw_train_data = input_directory + '/subtaskA_train_monolingual.jsonl'
raw_dev_data = input_directory + '/subtaskA_dev_monolingual.jsonl'

Mounted at /content/drive


In [23]:
# update with path to location of saved numpy array files generated by probability encoder notebook
train_encoded_probabilities = np.load("/content/drive/MyDrive/2023-2024 School Year/Fall Semester/Natural Language Processing/Project/train_final_encoded_probabilities.npy")
dev_encoded_probabilities = np.load("/content/drive/MyDrive/2023-2024 School Year/Fall Semester/Natural Language Processing/Project/dev_final_encoded_probabilities.npy")

In [20]:
import json

def extract_labels(filename):
  labels = []
  with open(filename, 'r', encoding='utf-8') as f:
    jlist = list(f)
    for elem in jlist:
      jsonData = json.loads(elem)
      labels.append(jsonData["label"])
  return labels

In [24]:
train_labels = extract_labels(raw_train_data)
dev_labels = extract_labels(raw_dev_data)

In [22]:
# if using train text for validation
train_encoded_probabilities, eval_encoded_probabilities, train_labels, eval_labels = train_test_split(train_encoded_probabilities, train_labels, test_size=0.1, random_state=92)

In [25]:
# if using dev set for validation
train_encoded_probabilities, train_labels = sk.utils.shuffle(train_encoded_probabilities, train_labels, random_state=92)
eval_encoded_probabilities = dev_encoded_probabilities
eval_labels = dev_labels

In [26]:
def build_model():
  inputs = tf.keras.Input(shape=(511, 2), dtype='float32')
  lstm = tf.keras.layers.LSTM(units = 10, activation="tanh", dropout=0.1)
  bidirectional = tf.keras.layers.Bidirectional(lstm)
  lstm_outputs = bidirectional(inputs)
  classifier = tf.keras.layers.Dense(1, activation='sigmoid')
  outputs = classifier(lstm_outputs)
  return tf.keras.Model(inputs, outputs)

In [27]:
final_train_data = train_encoded_probabilities
final_train_labels = np.array(train_labels)
final_test_data = eval_encoded_probabilities
final_test_labels = np.array(eval_labels)


In [28]:
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath='checkpointbest', save_best_only=True, save_weights_only=True, monitor="val_loss", mode="min")

In [ ]:
model = build_model()
#optimizer
batch_size = 100
epochs = 10
steps_per_epoch = int(final_train_data.shape[0] / batch_size)
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = int(0.1*num_train_steps)

init_lr = 0.001
optimizer = optimization.create_optimizer(init_lr=init_lr,
                                          num_train_steps=num_train_steps,
                                          num_warmup_steps=num_warmup_steps,
                                          optimizer_type='adamw')
#loss
loss= tf.keras.losses.BinaryCrossentropy(from_logits=False)
#metrics
metrics=[
    tfa.metrics.F1Score(num_classes=1, average="micro", name='micro_f1', threshold=0.5),
    tf.metrics.BinaryAccuracy()
]
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

model.fit(x=final_train_data, y=final_train_labels, validation_data = (final_test_data, final_test_labels), batch_size=batch_size, epochs=epochs, callbacks=[checkpoint])

In [ ]:
model.load_weights('checkpointbest')
model.evaluate(x=final_test_data, y=final_test_labels, batch_size=batch_size)